In [1]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F

In [5]:
wine_data = pd.read_csv("data/wine_preprocessed.csv")

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

wine_data["Class"] = le.fit_transform(wine_data["Class"])

In [7]:
wine_features = wine_data.drop("Class", axis=1)

In [8]:
wine_target = wine_data[["Class"]]

,Class
69,1
90,1
162,2
161,2
15,0


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wine_features, wine_target, test_size=0.2, random_state=42)

In [11]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()

In [12]:
ytrain_ = torch.from_numpy(y_train.values).view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).view(1, -1)[0]

In [13]:
input_size = 13
output_size = 3
hidden_size = 100

In [14]:
class StupidNet(nn.Module):
    def __init__(self):
        super(StupidNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=-1)

In [15]:
model = StupidNet()

import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)

loss_fn = nn.NLLLoss()

In [16]:
epochs = 1000

In [17]:
for epoch in range(epochs):
    
    optimizer.zero_grad()
    ypred_ = model(Xtrain_)
    
    loss = loss_fn(ypred_, ytrain_)
    
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch", epoch, "loss", loss.item())

Epoch 0 loss 1.1285029649734497
Epoch 100 loss 0.24380402266979218
Epoch 200 loss 0.05995741859078407
Epoch 300 loss 0.040698084980249405
Epoch 400 loss 0.21363332867622375
Epoch 500 loss 0.10048705339431763
Epoch 600 loss 0.048771876841783524
Epoch 700 loss 0.04548053443431854
Epoch 800 loss 0.1694325953722
Epoch 900 loss 0.06430227309465408


In [18]:
sample = np.array(X_test)
sample_tensor = torch.from_numpy(sample).float()

In [20]:
torch.onnx.export(model, sample_tensor, "models/classifier.onnx", export_params=True)

In [26]:
import onnx
import caffe2.python.onnx.backend as onnx_caffe2_backend

In [27]:
new_model = onnx.load("models/classifier.onnx")

In [29]:
prepare_backend = onnx_caffe2_backend.prepare(new_model)

In [30]:
new_model.graph.input[0]

name: "input.1"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 36
      }
      dim {
        dim_value: 13
      }
    }
  }
}

In [33]:
test_df = pd.read_csv("data/wine_test_data.csv")

In [34]:
X_test = test_df.drop("Class", axis=1)

In [36]:
y_test = test_df["Class"]

In [37]:
sample = np.array(X_test)
sample_tensor = torch.from_numpy(sample).float()

In [38]:
W = {new_model.graph.input[0].name: sample_tensor.numpy()}

In [41]:
c2_out = prepare_backend.run(W)

In [43]:
output_tensor = torch.Tensor(c2_out[0])

In [45]:
pred = torch.argmax(output_tensor, dim=-1)

In [46]:
pred

tensor([0, 1, 0, 2, 2, 0, 1, 2, 1, 1])

In [50]:
predicted_df = pd.DataFrame(data=pred.numpy(), columns= ["Predicted Class"])

In [53]:
compare = [predicted_df, y_test]
compare_df = pd.concat(compare, axis=1)

In [54]:
compare_df

,Predicted Class,Class
0,0,0
1,1,1
2,0,0
3,2,2
4,2,2
5,0,0
6,1,1
7,2,2
8,1,1
9,1,1
